# OSP model

Changes:
- Absolute path
- Flexible zero-error-radius
- Examine output by teaching signal and slots

## Import libraries

In [ ]:
%load_ext lab_black
import h5py, pickle, os
import tensorflow as tf
import numpy as np
import pandas as pd

import meta, data_wrangling, modeling, evaluate
from IPython.display import clear_output

meta.gpu_mem_cap(2048)  # Put memory cap to allow parallel runs
meta.check_gpu()

## Parameters block for Papermill
- Instead of using model_cfg directly, this extra step is needed for batch run using Papermill
- Consider carefully the variable type in each cfg setting (Probably automatically check it later...)
    - Do not use integer (e.g., 1, 2, 3, 0) in variables that can be float32 (e.g., w_oh_noise, tau...)
    - Use integer with a dot instead (e.g., 1., 2., 3., 0.)
- To use attractor, two params must be config, 
    - 1) embed_attractor_cfg --> json cfg file of the pretrain attractor 
    - 2) embed_attractor_h5 --> h5 file of the exact weight (e.g. ep0500.h5 #epoch or c90.h5 #correct rate)

In [ ]:
code_name = "boox"

sample_name = "jay"
rng_seed = 53797
use_semantic = False

# Model architechture
input_dim = 119
output_dim = 250
hidden_units = 100
cleanup_units = 20

pretrain_attractor = False  # Load pretrained or not

# embed_attractor_cfg = 'models/Attractor_{0:02d}/model_config.json'.format(
#     cleanup_units
# )
# embed_attractor_h5 = 'c00.h5'

rnn_activation = "sigmoid"
regularizer_const = None
w_initializer = 0.1  # range of uniform random
zero_error_radius = 0.5  # When True, zer value = 0.1, hardcoded in modeling.zer_bce()


p_noise = 0.0
tau = 1 / 3
max_unit_time = 4.0
output_ticks = 2

# Training
optimizer = "adam"
n_mil_sample = 1.0
batch_size = 128
learning_rate = 0.05
save_freq = 10

bq_dataset = None
batch_unique_setting_string = None

## Construct model configuration

In [ ]:
d = {}
for v in meta.model_cfg.minimal_cfgs:
    d[v] = globals()[v]

for v in meta.model_cfg.aux_cfgs:
    try:
        d[v] = globals()[v]
    except:
        pass

cfg = meta.model_cfg(**d)

tf.random.set_seed(cfg.rng_seed)
data = data_wrangling.my_data(cfg)

# Modeling

### Custom metrics (in development)

In [ ]:
class MeanOutputSlot10(tf.keras.metrics.Metric):
    """Export last slot average output in last batch of a epoch
    """

    def __init__(self, name="mean_output_slot10", **kwargs):
        super(MeanOutputSlot10, self).__init__(name=name, **kwargs)
        self.out10 = self.add_weight(name="out10", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred_slots = tf.split(y_pred, 10, axis=-1)
        self.out10.assign(tf.reduce_mean(y_pred_slots[9]))

    def result(self):
        return self.out10

    def reset_states(self):
        # The state of the metric will be reset at the start of each epoch.
        self.out10.assign(0.0)


class MeanOutputSlot4(tf.keras.metrics.Metric):
    """Export last slot average output in last batch of a epoch
    """

    def __init__(self, name="mean_output_slot4", **kwargs):
        super(MeanOutputSlot4, self).__init__(name=name, **kwargs)
        self.out4 = self.add_weight(name="out4", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred_slots = tf.split(y_pred, 10, axis=-1)
        self.out4.assign(tf.reduce_mean(y_pred_slots[3]))

    def result(self):
        return self.out4

    def reset_states(self):
        # The state of the metric will be reset at the start of each epoch.
        self.out4.assign(0.0)

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Layer, Input, concatenate, multiply, RepeatVector
from tensorflow.keras.optimizers import Adam, SGD


def build_model(training=True):
    """
    Create Keras model
    Note that:
    For structural things, such as repeat vector, should build within the model
    For Static calculation of input, it is easier to modify, should build within sample generator
    """

    cfg.noise_on() if training else cfg.noise_off()

    input_o = Input(shape=(cfg.input_dim,), name="Input_O")
    input_o_t = RepeatVector(cfg.n_timesteps, name="Input_Ot")(input_o)

    # Construct semantic input
    if cfg.use_semantic == True:
        raw_s_t = Input(shape=(cfg.n_timesteps, cfg.output_dim), name="Plaut_St")

        input_p = Input(shape=(cfg.output_dim,), name="input_P")
        input_p_t = RepeatVector(cfg.n_timesteps, name="Teaching_Pt")(input_p)

        input_s_t = multiply([raw_s_t, input_p_t], name="Input_St")

        combined = concatenate([input_o_t, input_s_t], name="Combined_input")
        rnn_model = rnn(cfg)(combined)
        model = Model([input_o, raw_s_t, input_p], rnn_model)

    else:
        rnn_model = modeling.rnn(cfg)(input_o_t)
        model = Model(input_o, rnn_model)

    # Select optimizer
    if cfg.optimizer == "adam":
        op = Adam(
            learning_rate=cfg.learning_rate, beta_1=0.9, beta_2=0.999, amsgrad=False
        )

    elif cfg.optimizer == "sgd":
        op = SGD(cfg.learning_rate)

    # Select zero error radius (by chossing custom loss function zer_bce())

    me = ["BinaryAccuracy", "mse", MeanOutputSlot4(), MeanOutputSlot10()]

    if cfg.zero_error_radius is not None:
        print(f"Using zero-error-radius of {zero_error_radius}")
        model.compile(
            loss=modeling.CustomBCE(radius=cfg.zero_error_radius),
            optimizer=op,
            metrics=me,
        )

    elif cfg.zero_error_radius is None:
        print(f"No zero-error-radius")
        model.compile(loss="binary_crossentropy", optimizer=op, metrics=me)

    model.summary()
    return model


model = build_model(training=True)

## Arming attractor

In [ ]:
if cfg.pretrain_attractor is True:
    print("Found attractor info in config (cfg), arming attractor...")
    attractor_cfg = meta.model_cfg(cfg.embed_attractor_cfg, bypass_chk=True)
    attractor_obj = modeling.attractor(attractor_cfg, cfg.embed_attractor_h5)
    model = modeling.arm_attractor(model, attractor_obj)
    evaluate.plot_variables(model)
else:
    print("Config indicates no attractor, I have do nothing.")

## Training

In [ ]:
checkpoint = modeling.ModelCheckpoint_custom(
    cfg.path_weights_checkpoint, save_weights_only=True, period=cfg.save_freq,
)

# tensorboard_callback = tf.keras.callbacks.TensorBoard(
#     cfg.path_log_folder, histogram_freq=1
# )

history = model.fit(
    data_wrangling.sample_generator(cfg, data),
    steps_per_epoch=cfg.steps_per_epoch,
    epochs=cfg.nEpo,
    verbose=0,
    callbacks=[checkpoint],
)


# Saving history and model
pickle_out = open(cfg.path_history_pickle, "wb")
pickle.dump(history.history, pickle_out)
pickle_out.close()

clear_output()
print("Training done")

# Parse results

### Parse item level stats

In [ ]:
# Must turn training mode off before evaluation
model = build_model(training=False)

In [ ]:
# Strain full model
strain = evaluate.strain_eval(cfg, data, model)
strain.start_evaluate(
    test_use_semantic=False, output=cfg.path_model_folder + "result_strain_item.csv"
)

# Semantic lesion in Strain
# if cfg.use_semantic == True:
#     strain_ns = strain_eval(cfg, data, model)
#     strain_ns.start_evaluate(
#         test_use_semantic=False,
#         output=cfg.path_model_folder + 'result_strain_ns_item.csv'
#     )

# Grain
grain = evaluate.grain_eval(cfg, data, model)
grain.start_evaluate(output=cfg.path_model_folder + "result_grain_item.csv")

# Taraban
taraban = evaluate.taraban_eval(cfg, data, model)
taraban.start_evaluate(
    test_use_semantic=False, output=cfg.path_model_folder + "result_taraban_item.csv"
)

# Glushko
glushko = evaluate.glushko_eval(cfg, data, model)
glushko.start_evaluate(False, cfg.path_model_folder + "result_glushko_item.csv")

In [ ]:
# Pack results into visualization class
vis = evaluate.vis(cfg.path_model_folder)

### Training history

In [ ]:
vis.training_hist().save(cfg.path_plot_folder + "training_history.html")

### Output at slot 4 & 10

In [ ]:
x = evaluate.training_history(cfg.path_history_pickle)
x.plot("rnn_1_mean_output_slot10", "output10").save(
    cfg.path_plot_folder + "output_slot10.html"
)

x.plot("rnn_1_mean_output_slot4", "output4").save(
    cfg.path_plot_folder + "output_slot4.html"
)

### Strain and Grain plots

In [ ]:
# vis_ns = vis(
#     cfg.path_model_folder, 'result_strain_ns_item.csv', 'result_grain_item.csv'
# )
# vis_ns.parse_cond_df()
# lesion = vis_ns.plot_dev('acc').properties(title='Semantic lesion')
# strain_plot = full | lesion
# strain_plot

sg = vis.plot_dev_interactive("acc", ["strain", "grain"]).properties(
    title="Accuracy in Strain and Grain"
)
sg.save(cfg.path_plot_folder + "development_sg.html")

### Lesion development deep dive

In [ ]:
# dev_inter = vis.plot_dev_interactive('acc')
# dev_inter.save(cfg.path_plot_folder + 'interactive_strain_dev_full.html')
# dev_inter

# dev_inter = vis_ns.plot_dev_interactive('acc')
# dev_inter.save(cfg.path_plot_folder + 'interactive_strain_dev_lesion.html')
# dev_inter

### Lesion time plot deep dive

In [ ]:
# time_inter = vis.plot_time_interactive('acc')
# time_inter.save(cfg.path_plot_folder + 'interactive_strain_time.html')
# time_inter

### Grain plots

In [ ]:
small = vis.plot_dev_interactive("acc_small_grain", exp=["grain"]).properties(
    title="Small Grain Response"
)
large = vis.plot_dev_interactive("acc_large_grain", exp=["grain"]).properties(
    title="Large Grain Response"
)
grain_plot = (small | large).properties(
    title="Accuracy of Grain by response and condition"
)
grain_plot.save(cfg.path_plot_folder + "development_grain_by_response.html")

### Taraban and Glushko plots

In [ ]:
tb = vis.plot_dev_interactive("acc", ["taraban", "glushko"]).properties(
    title="Accuracy in Taraban and Glushko by condition"
)
tb.save(cfg.path_plot_folder + "development_tg.html")

### Words vs. Nonwords

In [ ]:
wnw_zr = vis.plot_wnw(["INC_HF"], ["unambiguous"]).properties(
    title="Strain (INC_HF) vs. Grain (UN)"
)

all_taraban_conds = list(vis.cdf.loc[vis.cdf.exp == "taraban", "cond"].unique())
wnw_tg = vis.plot_wnw(all_taraban_conds, ["Exception", "Regular"]).properties(
    title="Taraban (all) vs. Glushko (NW)"
)
wnw_plot = wnw_zr | wnw_tg
wnw_plot.save(cfg.path_plot_folder + "wnw.html")

### Model weights and biases

In [ ]:
evaluate.weight(cfg.path_weights_list[-1]).violinplot(
    cfg.path_plot_folder + "weight_violin.png"
)

In [ ]:
evaluate.weight(cfg.path_weights_list[-1]).heatmap(
    cfg.path_plot_folder + "weight_heatmap.png"
)

In [ ]:
# !jupyter nbconvert --output-dir=$cfg.path_model_folder --to html OSP_master.ipynb